In [1]:
from sympy.physics.vector import (
    ReferenceFrame,
    dynamicsymbols,
    get_motion_params
)
from sympy import symbols, Piecewise, Eq, solveset, Min, Max, Abs, latex

from IPython.display import Markdown
import numpy as np
import plotly.graph_objects as go

# Reference frame

In [2]:
N = ReferenceFrame('N')

In [3]:
Markdown(rf"""
$$N = \{{ {latex(N.x)}, {latex(N.y)}, {latex(N.z)} \}}$$
""")


$$N = \{ \mathbf{\hat{n}_x}, \mathbf{\hat{n}_y}, \mathbf{\hat{n}_z} \}$$


# Reference variables

In [4]:
T = symbols('T', nonnegative=True)
R = symbols('R', nonnegative=True)
t = symbols('t', nonnegative=True)

In [5]:
a_magnitude = symbols('|a|', nonnegative=True)
v0_magnitude = symbols('|v_{0}|', nonnegative=True) # TODO: v0 should be a 3D vector, and accept negative values
p0_magnitude = symbols('|p_{0}|', nonnegative=True) # TODO: d0 should be a 3D vector, and accept negative values

a_vector = N.x * Piecewise((-a_magnitude, T * R - dynamicsymbols._t < 0), (a_magnitude, True))
# a_vector = N.x * a_magnitude * (T * R - dynamicsymbols._t) / Abs(T * R - dynamicsymbols._t)
v0_vector = N.x * v0_magnitude
p0_vector = N.x * p0_magnitude

pT_magnitude = symbols('|d_{T}|', nonnegative=True) # TODO: pT should be a 3D vector, and accept negative values
pT_vector = N.x * pT_magnitude

vT_magnitude = symbols('|v_{T}|')
vT_vector = N.x * vT_magnitude

In [6]:
Markdown(rf"""
$$
\begin{{align*}}
|a| & \quad & m.s^{{-2}} \quad & \text{{ Acceleration magnitude}} \\
v_0 &= {latex(v0_vector)} \quad & m.s^{{-1}} \quad & \text{{ Velocity at }} t_0 \\
v_T &= {latex(vT_vector)} \quad & m.s^{{-1}} \quad & \text{{ Velocity at destination}} \\
p_0 &= {latex(p0_vector)} \quad & m \quad & \text{{ Position at }} t_0 \\
p_T &= {latex(pT_vector)} \quad & m \quad & \text{{ Position at destination}}
\end{{align*}}
$$
""")


$$
\begin{align*}
|a| & \quad & m.s^{-2} \quad & \text{ Acceleration magnitude} \\
v_0 &= |v_{0}|\mathbf{\hat{n}_x} \quad & m.s^{-1} \quad & \text{ Velocity at } t_0 \\
v_T &= |v_{T}|\mathbf{\hat{n}_x} \quad & m.s^{-1} \quad & \text{ Velocity at destination} \\
p_0 &= |p_{0}|\mathbf{\hat{n}_x} \quad & m \quad & \text{ Position at } t_0 \\
p_T &= |d_{T}|\mathbf{\hat{n}_x} \quad & m \quad & \text{ Position at destination}
\end{align*}
$$


# Motion parameters, with constant acceleration  𝑎

In [7]:
def replace_max_0_RT(f):
    # Given that T >= 0 and R in [0, 1]:
    #  We can say that max(0, R * T) = R * T
    #  However sympy is not able to infer this, so we replace manually
    return f.replace(Max, lambda a, b: (b if a == 0 else a_))

In [8]:
original_a_fn, original_v_fn, original_d_fn = get_motion_params(
    N,
    acceleration=a_vector, 
    velocity=v0_vector,
    position=p0_vector
)

a_fn = original_a_fn.subs(dynamicsymbols._t, t)
v_fn = original_v_fn.subs(dynamicsymbols._t, t)#.applyfunc(replace_min_T)
d_fn = original_d_fn.subs(dynamicsymbols._t, t).applyfunc(replace_max_0_RT)

In [9]:
Markdown(rf"""
These formulae depend on:
* $T$ (in seconds): The total time to arrival
* $R \in [0, 1]$: The ratio at which we start decelerating

$$
\begin{{align}}
a(t) &= {latex(a_fn)} \\
v(t) &= {latex(v_fn)} \\
d(t) &= {latex(d_fn)}
\end{{align}}
$$
""")


These formulae depend on:
* $T$ (in seconds): The total time to arrival
* $R \in [0, 1]$: The ratio at which we start decelerating

$$
\begin{align}
a(t) &= \begin{cases} - |a| & \text{for}\: R T - t < 0 \\|a| & \text{otherwise} \end{cases}\mathbf{\hat{n}_x} \\
v(t) &= (- t |a| + 2 |a| \min\left(t, R T\right) + |v_{0}|)\mathbf{\hat{n}_x} \\
d(t) &= (- \frac{t^{2} |a|}{2} + t \left(2 R T |a| + |v_{0}|\right) + |a| \min\left(t, R T\right)^{2} + |p_{0}| + |v_{0}| \min\left(t, R T\right) - \left(2 R T |a| + |v_{0}|\right) \min\left(t, R T\right))\mathbf{\hat{n}_x}
\end{align}
$$


# Finding R

In [10]:
def replace_min_T(f):
    # Given that T >= 0 and R in [0, 1]:
    #  We can say that min(t, R * T) = R * T
    #  However sympy is not able to infer this, so we replace manually
    return f.replace(Min(T, R*T), R*T)

In [11]:
eq_vT_x = Eq(
    vT_vector.to_matrix(N)[0],
    v_fn.subs(t, T).applyfunc(replace_min_T).to_matrix(N)[0]
) # TODO: Make that equation on 3 dimensions

solved_R = list(solveset(eq_vT_x, R))[0] # The solveset returns a singleton, which is also a list of size 1

In [12]:
Markdown(rf"""
$R \in [0, 1]$: The ratio at which we start decelerating

$$
\begin{{align}}
v_T &= v(T) \\
{latex(vT_vector.to_matrix(N)[0])} &= {latex(v_fn.subs(t, T).to_matrix(N)[0])} \\
\end{{align}}
$$

This solves into:

$$R = {latex(solved_R)}$$
""")


$R \in [0, 1]$: The ratio at which we start decelerating

$$
\begin{align}
v_T &= v(T) \\
|v_{T}| &= - T |a| + 2 |a| \min\left(T, R T\right) + |v_{0}| \\
\end{align}
$$

This solves into:

$$R = \frac{T |a| - |v_{0}| + |v_{T}|}{2 T |a|}$$


# Finding T

In [13]:
eq_dT_x = Eq(
    pT_vector.to_matrix(N)[0],
    d_fn.subs({t: T}).applyfunc(replace_min_T).subs({R: solved_R}).to_matrix(N)[0]
) # TODO: Make that equation on 3 dimensions

# eq_dT_x solves as a quadratic expression, thus returns a finite set of two values.
solved_T1, solved_T2 = solveset(eq_dT_x, T)
solved_T = Max(solved_T1, solved_T2)

In [14]:
Markdown(rf"""
$T$ (in seconds): The total time to arrival

$$
\begin{{align}}
p_T &= p(T) \\
{latex(pT_vector.to_matrix(N)[0])} &= {latex(d_fn.subs({t: T}).to_matrix(N)[0])} \\
\\
\text{{Given that }} R &= {latex(solved_R)} \\
{latex(pT_vector.to_matrix(N)[0])} &= {latex(d_fn.subs({t: T, R: solved_R}).to_matrix(N)[0])} \\
\end{{align}}
$$

This solves into:

$$T = {latex(solved_T)}$$
""")


$T$ (in seconds): The total time to arrival

$$
\begin{align}
p_T &= p(T) \\
|d_{T}| &= - \frac{T^{2} |a|}{2} + T \left(2 R T |a| + |v_{0}|\right) + |a| \min\left(T, R T\right)^{2} + |p_{0}| + |v_{0}| \min\left(T, R T\right) - \left(2 R T |a| + |v_{0}|\right) \min\left(T, R T\right) \\
\\
\text{Given that } R &= \frac{T |a| - |v_{0}| + |v_{T}|}{2 T |a|} \\
|d_{T}| &= - \frac{T^{2} |a|}{2} + T \left(T |a| + |v_{T}|\right) + |a| \min\left(T, \frac{T |a| - |v_{0}| + |v_{T}|}{2 |a|}\right)^{2} + |p_{0}| + |v_{0}| \min\left(T, \frac{T |a| - |v_{0}| + |v_{T}|}{2 |a|}\right) - \left(T |a| + |v_{T}|\right) \min\left(T, \frac{T |a| - |v_{0}| + |v_{T}|}{2 |a|}\right) \\
\end{align}
$$

This solves into:

$$T = \max\left(- \frac{|v_{0}| + |v_{T}|}{|a|} - \frac{\sqrt{2} \sqrt{2 |a| |d_{T}| - 2 |a| |p_{0}| + |v_{0}|^{2} + |v_{T}|^{2}}}{|a|}, - \frac{|v_{0}| + |v_{T}|}{|a|} + \frac{\sqrt{2} \sqrt{2 |a| |d_{T}| - 2 |a| |p_{0}| + |v_{0}|^{2} + |v_{T}|^{2}}}{|a|}\right)$$


# Motion parameters, expand equations

In [15]:
expanded_a_fn = a_fn.subs({T: solved_T, R: solved_R})
expanded_v_fn_2 = v_fn.subs({T: solved_T, R: solved_R})
expanded_d_fn_2 = d_fn.subs({T: solved_T, R: solved_R})

In [20]:
Markdown(rf"""
After injecting $T$ and $R$, we obtain:

$$
\begin{{align}}
a(t) &= {latex(expanded_a_fn)} \\
v(t) &= {latex(expanded_v_fn_2)} \\
d(t) &= {latex(expanded_d_fn_2)}
\end{{align}}
$$

**Remark:** These results are not useful in practice, and are just written here as an indication
""")


After injecting $T$ and $R$, we obtain:

$$
\begin{align}
a(t) &= \begin{cases} - |a| & \text{for}\: t - \frac{|a| \max\left(- \frac{|v_{0}| + |v_{T}|}{|a|} - \frac{\sqrt{2} \sqrt{2 |a| |d_{T}| - 2 |a| |p_{0}| + |v_{0}|^{2} + |v_{T}|^{2}}}{|a|}, - \frac{|v_{0}| + |v_{T}|}{|a|} + \frac{\sqrt{2} \sqrt{2 |a| |d_{T}| - 2 |a| |p_{0}| + |v_{0}|^{2} + |v_{T}|^{2}}}{|a|}\right) - |v_{0}| + |v_{T}|}{2 |a|} > 0 \\|a| & \text{otherwise} \end{cases}\mathbf{\hat{n}_x} \\
v(t) &= (- t |a| + 2 |a| \min\left(t, \frac{|a| \max\left(- \frac{|v_{0}| + |v_{T}|}{|a|} - \frac{\sqrt{2} \sqrt{2 |a| |d_{T}| - 2 |a| |p_{0}| + |v_{0}|^{2} + |v_{T}|^{2}}}{|a|}, - \frac{|v_{0}| + |v_{T}|}{|a|} + \frac{\sqrt{2} \sqrt{2 |a| |d_{T}| - 2 |a| |p_{0}| + |v_{0}|^{2} + |v_{T}|^{2}}}{|a|}\right) - |v_{0}| + |v_{T}|}{2 |a|}\right) + |v_{0}|)\mathbf{\hat{n}_x} \\
d(t) &= (- \frac{t^{2} |a|}{2} + t \left(|a| \max\left(- \frac{|v_{0}| + |v_{T}|}{|a|} - \frac{\sqrt{2} \sqrt{2 |a| |d_{T}| - 2 |a| |p_{0}| + |v_{0}|^{2} + |v_{T}|^{2}}}{|a|}, - \frac{|v_{0}| + |v_{T}|}{|a|} + \frac{\sqrt{2} \sqrt{2 |a| |d_{T}| - 2 |a| |p_{0}| + |v_{0}|^{2} + |v_{T}|^{2}}}{|a|}\right) + |v_{T}|\right) + |a| \min\left(t, \frac{|a| \max\left(- \frac{|v_{0}| + |v_{T}|}{|a|} - \frac{\sqrt{2} \sqrt{2 |a| |d_{T}| - 2 |a| |p_{0}| + |v_{0}|^{2} + |v_{T}|^{2}}}{|a|}, - \frac{|v_{0}| + |v_{T}|}{|a|} + \frac{\sqrt{2} \sqrt{2 |a| |d_{T}| - 2 |a| |p_{0}| + |v_{0}|^{2} + |v_{T}|^{2}}}{|a|}\right) - |v_{0}| + |v_{T}|}{2 |a|}\right)^{2} + |p_{0}| + |v_{0}| \min\left(t, \frac{|a| \max\left(- \frac{|v_{0}| + |v_{T}|}{|a|} - \frac{\sqrt{2} \sqrt{2 |a| |d_{T}| - 2 |a| |p_{0}| + |v_{0}|^{2} + |v_{T}|^{2}}}{|a|}, - \frac{|v_{0}| + |v_{T}|}{|a|} + \frac{\sqrt{2} \sqrt{2 |a| |d_{T}| - 2 |a| |p_{0}| + |v_{0}|^{2} + |v_{T}|^{2}}}{|a|}\right) - |v_{0}| + |v_{T}|}{2 |a|}\right) - \left(|a| \max\left(- \frac{|v_{0}| + |v_{T}|}{|a|} - \frac{\sqrt{2} \sqrt{2 |a| |d_{T}| - 2 |a| |p_{0}| + |v_{0}|^{2} + |v_{T}|^{2}}}{|a|}, - \frac{|v_{0}| + |v_{T}|}{|a|} + \frac{\sqrt{2} \sqrt{2 |a| |d_{T}| - 2 |a| |p_{0}| + |v_{0}|^{2} + |v_{T}|^{2}}}{|a|}\right) + |v_{T}|\right) \min\left(t, \frac{|a| \max\left(- \frac{|v_{0}| + |v_{T}|}{|a|} - \frac{\sqrt{2} \sqrt{2 |a| |d_{T}| - 2 |a| |p_{0}| + |v_{0}|^{2} + |v_{T}|^{2}}}{|a|}, - \frac{|v_{0}| + |v_{T}|}{|a|} + \frac{\sqrt{2} \sqrt{2 |a| |d_{T}| - 2 |a| |p_{0}| + |v_{0}|^{2} + |v_{T}|^{2}}}{|a|}\right) - |v_{0}| + |v_{T}|}{2 |a|}\right))\mathbf{\hat{n}_x}
\end{align}
$$

**Remark:** These results are not useful in practice, and are just written here as an indication


# Plotting

In [18]:
import ipywidgets as widgets
from ipywidgets import interactive_output

a_knob = widgets.BoundedFloatText(value=1, min=0.1, max=15.0, step=0.1, description='Acceleration (m.s-2)')
v0_knob = widgets.IntText(value=0, step=10, description='v0 (m.s-1)')
vT_knob = widgets.IntText(value=0, step=10, description='vT (m.s-1)')
p0_knob = widgets.IntText(value=0, step=10, description='p0 (m)')
pT_knob = widgets.IntText(value=100000, step=10, description='pT (m)')

ui = widgets.GridBox([a_knob, v0_knob, vT_knob, p0_knob, pT_knob])
display(ui)

GridBox(children=(BoundedFloatText(value=1.0, description='Acceleration (m.s-2)', max=15.0, min=0.1, step=0.1)…

In [35]:
from sympy.utilities.lambdify import lambdify

def make_plot(name, short_name, fn, args, line_shape="spline"):
    v_args = {a_magnitude: args["a_knob"], v0_magnitude: args["v0_knob"], vT_magnitude: args["vT_knob"]}
    R_val = solved_R.subs(v_args)
    
    p_args = {**v_args, p0_magnitude: args["p0_knob"], pT_magnitude: args["pT_knob"]}
    T_val = solved_T.subs(p_args)
    
    t_sample = np.linspace(0, float(T_val), 99)
    image_sample = lambdify(t, fn.subs({**p_args, T: T_val, R: R_val}).to_matrix(N)[0])
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=list(t_sample), y=[image_sample(t_val) for t_val in t_sample], line_shape=line_shape, name=name))

    fig.update_layout(title=name, xaxis_title="t", yaxis_title=short_name)
    return fig

def plot_position(**knob_args):
    plot = make_plot("Position [x])", "p_x(t)", d_fn, knob_args)
    plot.show()
    
def plot_velocity(**knob_args):
    plot = make_plot("Velocity [x])", "v_x(t)", v_fn, knob_args)
    plot.show()
    
def plot_acceleration(**knob_args):
    plot = make_plot("Acceleration [x])", "a_x(t)", a_fn, knob_args, line_shape="hv")
    plot.show()

In [36]:
knob_args = {
    "a_knob": a_knob,
    "v0_knob": v0_knob, "vT_knob": vT_knob,
    "p0_knob": p0_knob, "pT_knob": pT_knob
}

position_plot = interactive_output(plot_position, knob_args)
velocity_plot = interactive_output(plot_velocity, knob_args)
acceleration_plot = interactive_output(plot_acceleration, knob_args)
display(position_plot, velocity_plot, acceleration_plot)

Output()

Output()

Output()